In [1]:
import os, sys
import pickle
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import torch
import pandas as pd
from datetime import datetime, timedelta
sys.path.insert(0,'../../LongTermEMG-master')

# Process Data

In [2]:
from LongTermClassificationMain.PrepareAndLoadDataLongTerm.prepare_dataset_utils import butter_bandpass_filter, \
    show_filtered_signal, load_timestamps_from_participant, get_angles_from_positions_3d_arm
from LongTermClassificationMain.PrepareAndLoadDataLongTerm import feature_extraction

In [3]:
os.chdir("/home/laiy/gitrepos/msr_final/LongTermEMG-master/LongTermClassificationMain/PrepareAndLoadDataLongTerm")
print(os.listdir("../../"))

['longterm_dataset_3DC.zip', 'longterm_dataset_3DC', 'README.md', 'LongTermClassificationMain', 'datasets', 'TransferLearning', '.idea']


In [4]:
list_participant_training_1_to_skip = ["Participant0/Training1", "Participant0/Evaluation2", "Participant0/Evaluation3",
                                       "Participant2/Training1", "Participant2/Evaluation2", "Participant2/Evaluation3"]

In [5]:
features_set_name = "TSD_features_set"
feature_set_function = feature_extraction.getTSD
path="../../datasets/longterm_dataset_3DC"
number_of_gestures=11
number_of_cycles=4
window_size=150 # 150ms frame with 100 overlap
size_non_overlap=50

In [6]:
def format_examples(emg_examples, feature_set_function, window_size=150, size_non_overlap=50):
    ''' 
    emg_examples: list of emg signals, each member represent one recording of a 10 channel emg
    
    feature_set_function:
        shape(formated_example) = (97, 10, 150)
        shape(featured_example) = (97, 385)
        might not need this for myo
    
    window_size, size_non_verlap: 150ms frame with 100ms overlap    
        ignoring leftover examples at the end
        
    '''
    examples_to_calculate_features_set_from = []
    example = []
    for emg_vector in emg_examples:
        if len(example) == 0:
            example = emg_vector
        else:
            example = np.row_stack((example, emg_vector))
        
        if len(example) >= window_size:
            # The example is of the shape TIME x CHANNEL. Make it of the shape CHANNEL x TIME
            example = example.transpose()
            # Go over each channel and bandpass filter it between 20 and 495 Hz.
            example_filtered = []
            for channel in example:
                channel_filtered = butter_bandpass_filter(channel, lowcut=20, highcut=495, fs=1000, order=4)
                # show_filtered_signal(channel, channel_filtered)
                example_filtered.append(channel_filtered)
            # Add the filtered example to the list of examples to return and transpose the example array again to go
            # back to TIME x CHANNEL
            examples_to_calculate_features_set_from.append(example_filtered)
            example = example.transpose()
            # Remove part of the data of the example according to the size_non_overlap variable
            example = example[size_non_overlap:]
    
#     print("   @ formatting example = ", np.shape(examples_to_calculate_features_set_from))
    
    examples_features_set_calculated = feature_extraction.get_dataset_with_features_set(
        dataset=examples_to_calculate_features_set_from, features_set_function=feature_set_function)
#     print("   @ featured example = ", np.shape(examples_features_set_calculated))

    return examples_features_set_calculated

In [7]:
from handcrafted_features_prepare_from_from_raw_dataset import get_highest_average_emg_window

def read_files_to_format_training_session(path_folder_examples, feature_set_function,
                                          number_of_cycles, number_of_gestures, window_size,
                                          size_non_overlap):
    '''
    when #cycle = 1, examples are recorded at highest activation
        
    '''
    examples_training, labels_training = [], []
    # Check if the folder is empty, if so, skip it
    # path_folder_examples = path + "/" + folder_participant + "/" + training_directory + "/EMG"
    if len(os.listdir(path_folder_examples)) == 0:
        return [], [], []
#     print(os.listdir(path_folder_examples))
#--------  filename = `3dc_EMG_gesture_<#cycle>_<#gesture>.txt`-------------
    highest_activation_per_gesture = []
    for cycle in range(number_of_cycles):
        # path_folder_examples = path + folder_participant + "/" + training_directory + "/EMG"
        # This one instance, the participant only recorded one cycle of training. Skip it
        for participant_session_to_skip in list_participant_training_1_to_skip:
            if participant_session_to_skip in path_folder_examples:
                return [], [], []
        path_emg = path_folder_examples + "/3dc_EMG_gesture_%d_" % cycle
        examples, labels = [], []
        for gesture_index in range(number_of_gestures):
            examples_to_format = []
            read_file = path_emg + '%d.txt' % gesture_index
#             print("READ FILE ", read_file)
            
            for line in open(read_file):
                #  strip() remove the "\n" character, split separate the data in a list. np.float
                #  transform each element of the list from a str to a float
                emg_signal = np.float32(line.strip().split(","))
                examples_to_format.append(emg_signal)
#             print("   example emg from file (to format)", np.shape(examples_to_format))
            
            if cycle == 1:  # This cycle is the second cycle and correspond to the highest effort baseline. Record it.
                if gesture_index == 0:
                    highest_activation_per_gesture.append(0)
                else:
                    highest_activation_per_gesture.append(get_highest_average_emg_window(
                        examples_to_format, window_for_moving_average=window_size))
                    
#             print("   start formating")
            examples_formatted = format_examples(examples_to_format,
                                                 feature_set_function=feature_set_function, window_size=window_size,
                                                 size_non_overlap=size_non_overlap)
            examples.extend(examples_formatted)
            labels.extend(np.ones(len(examples_formatted)) * gesture_index)
#             print("--- format complete for gesture ", gesture_index, " #examples = ", len(examples))
            
        print("   SHAPE SESSION ", cycle, " EXAMPLES: ", np.shape(examples))
        examples_training.append(examples)
        labels_training.append(labels)
        print("   SHAPE ALL SESSION EXAMPLES: ", np.shape(examples_training))

    return examples_training, labels_training, highest_activation_per_gesture

In [8]:
examples_training_sessions_datasets, labels_training_sessions_datasets = [], []
highest_activation_participants = []
examples_evaluation_sessions_datasets, labels_evaluation_sessions_datasets, timestamps_emg_evaluation = [], [], []
angles_with_timestamps_emg_evaluation = []

training_datetimes, evaluation_datetimes = [], []

### Each participant has four training sessions, each session has four cycles

In [9]:
for index_participant in range(1):
    folder_participant = "/Participant" + str(index_participant)
    sessions_directories = os.listdir(path + folder_participant)
    display("sessions_directories", sessions_directories)
    
#     Don't need datetime
#     training_datetime, evaluation_datetime = load_timestamps_from_participant(path + folder_participant)
#     training_datetimes.append(training_datetime)
#     display("training_datetime ", training_datetime)
#     evaluation_datetimes.append(evaluation_datetime)
#     display("evaluation_datetime ", evaluation_datetime)

    examples_participant_training_sessions, labels_participant_training_sessions = [], []
    highest_activation_per_session = []
    examples_participant_evaluation_sessions, labels_participant_evaluation_sessions = [], []
    timestamps_evaluation_participant_sessions, angles_with_timestamps_participant_sessions = [], []
    
    
    """
    Don't need Evaluation part for now, only process Training 
    """

    for session_directory in sessions_directories:
        print("current dir = ", session_directory)

        if "Training" in session_directory:
            path_folder_examples = path + "/" + folder_participant + "/" + session_directory + "/EMG"
            examples_training, labels_training, highest_activation_per_gesture = \
                read_files_to_format_training_session(path_folder_examples=path_folder_examples,
                                                      number_of_cycles=number_of_cycles,
                                                      number_of_gestures=number_of_gestures,
                                                      window_size=window_size,
                                                      size_non_overlap=size_non_overlap,
                                                      feature_set_function=feature_set_function)
            if len(examples_training) > 0:
                # These instances, the participant only recorded one cycle of training. Skip it
                skip_it = False
                for participant_session_to_skip in list_participant_training_1_to_skip:
                    if participant_session_to_skip in path_folder_examples:
                        skip_it = True
                if skip_it is False:
                    examples_participant_training_sessions.append(examples_training)
                    labels_participant_training_sessions.append(labels_training)
                    highest_activation_per_session.append(highest_activation_per_gesture)

                    print("@ traning sessions = ", np.shape(examples_participant_training_sessions))

'sessions_directories'

['Evaluation3',
 'Training3',
 'Training1',
 'Training5',
 'Evaluation8',
 'Evaluation0',
 'Evaluation6',
 'Evaluation7',
 'Training0',
 'Evaluation2',
 'Training4',
 'Evaluation9',
 'Training2',
 'Evaluation1',
 'Evaluation4',
 'Evaluation5']

current dir =  Evaluation3
current dir =  Training3
   SHAPE SESSION  0  EXAMPLES:  (1009, 385)
   SHAPE ALL SESSION EXAMPLES:  (1, 1009, 385)
   SHAPE SESSION  1  EXAMPLES:  (1052, 385)
   SHAPE ALL SESSION EXAMPLES:  (2,)


/home/laiy/.local/lib/python3.8/site-packages/numpy/core/_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)


   SHAPE SESSION  2  EXAMPLES:  (949, 385)
   SHAPE ALL SESSION EXAMPLES:  (3,)
   SHAPE SESSION  3  EXAMPLES:  (1014, 385)
   SHAPE ALL SESSION EXAMPLES:  (4,)
@ traning sessions =  (1, 4)
current dir =  Training1
current dir =  Training5
current dir =  Evaluation8
current dir =  Evaluation0
current dir =  Evaluation6
current dir =  Evaluation7
current dir =  Training0
   SHAPE SESSION  0  EXAMPLES:  (1064, 385)
   SHAPE ALL SESSION EXAMPLES:  (1, 1064, 385)
   SHAPE SESSION  1  EXAMPLES:  (1046, 385)
   SHAPE ALL SESSION EXAMPLES:  (2,)
   SHAPE SESSION  2  EXAMPLES:  (1063, 385)
   SHAPE ALL SESSION EXAMPLES:  (3,)
   SHAPE SESSION  3  EXAMPLES:  (1017, 385)
   SHAPE ALL SESSION EXAMPLES:  (4,)
@ traning sessions =  (2, 4)
current dir =  Evaluation2
current dir =  Training4
   SHAPE SESSION  0  EXAMPLES:  (1067, 385)
   SHAPE ALL SESSION EXAMPLES:  (1, 1067, 385)
   SHAPE SESSION  1  EXAMPLES:  (1087, 385)
   SHAPE ALL SESSION EXAMPLES:  (2,)
   SHAPE SESSION  2  EXAMPLES:  (1067, 3

In [23]:
np.shape(labels_training_sessions_datasets[0][0][0])

(1009,)

In [10]:
# store processed data to dictionary
print('traning examples ', np.shape(examples_participant_training_sessions))
examples_training_sessions_datasets = []
examples_training_sessions_datasets.append(examples_participant_training_sessions)
print('all traning examples ', np.shape(examples_training_sessions_datasets))

print('traning labels ', np.shape(labels_participant_training_sessions))
labels_training_sessions_datasets = []
labels_training_sessions_datasets.append(labels_participant_training_sessions)
print('all traning labels ', np.shape(labels_training_sessions_datasets))

dataset_dictionnary = {"examples_training": np.array(examples_training_sessions_datasets, dtype=object),
                       "labels_training": np.array(labels_training_sessions_datasets, dtype=object)}

traning examples  (4, 4)
all traning examples  (1, 4, 4)
traning labels  (4, 4)
all traning labels  (1, 4, 4)


In [11]:
# store dictionary to pickle
training_session_dataset_dictionnary = {}
training_session_dataset_dictionnary["examples_training"] = dataset_dictionnary["examples_training"]
training_session_dataset_dictionnary["labels_training"] = dataset_dictionnary["labels_training"]

os.chdir("/home/laiy/gitrepos/msr_final/LongTermEMG-master/LongTermClassificationMain/PrepareAndLoadDataLongTerm")
with open("../Processed_datasets/%s_training_session.pickle" % features_set_name, 'wb') as f:
    pickle.dump(training_session_dataset_dictionnary, f, pickle.HIGHEST_PROTOCOL)

# Train TSD_DNN

In [13]:
os.chdir("/home/laiy/gitrepos/msr_final/LongTermEMG-master/LongTermClassificationMain/TrainingsAndEvaluations/ForTrainingSessions/TSD_DNN")
print(os.listdir("../../"))
with open("../../../Processed_datasets/TSD_features_set_training_session.pickle", 'rb') as f:
    dataset_training = pickle.load(file=f)

examples_datasets_train = dataset_training['examples_training']
print('traning examples ', np.shape(examples_datasets_train))
labels_datasets_train = dataset_training['labels_training']
print('traning labels ', np.shape(labels_datasets_train))

['self_learning', 'utils_training_and_evaluation.py', 'training_loops_preparations.py', 'ForTrainingSessions', 'test_polar_plot.py', '__pycache__', 'ForEvaluationSessions']
traning examples  (1, 4, 4)
traning labels  (1, 4, 4)


In [32]:
features_set_name

'TSD_features_set'